# Priors and Forward Model

In [ ]:
from importlib import reload 
import numpy as np
import h5py
from utils import *
import time

## Prior parameter ##
nl_level = 10

PRIOR = np.ones(4, dtype=[('MU', float, (4)), ('C', float, (4,4))])

mu = np.ones([4,4])
mu[0] = [8.5948, 8.0356, 0.9613, 9.5561]
mu[1] = [8.4563, 7.8829, 0.9186, 9.3749]
mu[2] = [8.6676, 8.0483, 1.0716, 9.7392]
mu[3] = [8.4136, 7.8345, 0.7348, 9.1484]
#print(mu)

c = np.ones([4,4,4])
c[0,:] = [[0.0037, 0.0027, 0.0008, 0.0000],
            [0.0027, 0.0030, 0.0008, 0.0000],
            [0.0008, 0.0008, 0.0003, 0.0000],
            [0.0000, 0.0000, 0.0000, 0.0111]]

c[1,:] = [[0.0032, 0.0028, 0.0011, 0.0000],
            [0.0028, 0.0033, 0.0010, 0.0000],
            [0.0011, 0.0010, 0.0006, 0.0000],
            [0.0000, 0.0000, 0.0000, 0.0096]]

c[2,:] = [[0.0023, 0.0016, 0.0008, 0.0000],
            [0.0016, 0.0017, 0.0007, 0.0000],
            [0.0008, 0.0007, 0.0006, 0.0000],
            [0.0000, 0.0000, 0.0000, 0.0070]]

c[3,:] = [[0.0008, 0.0005, 0.0003, 0.0000],
            [0.0005, 0.0004, 0.0002, 0.0000],
            [0.0003, 0.0002, 0.0003, 0.0000],
            [0.0000, 0.0000, 0.0000, 0.0023]]
#print(c)

wavelet_new = [-1039.55055580727,
            -4026.57620068033,
            -8194.18572794536,
            -11356.0387947636,
            -11906.6435543803,
            -10191.4389229172,
            -8639.29324618705,
            -10474.3682826362,
            -17093.5087710456,
            -25733.4288332681,
            -29906.2345220425,
            -23088.0727988944,
            -3091.80943987738,
            25306.1208726887,
            51180.4184438887,
            63288.5822013372,
            53619.9099732780,
            32940.2018555759,
            11743.5370194762,
            -3175.56289063239,
            -10845.0261972202,
            -13246.6910319655,
            -12569.5536619989,
            -10329.2514738468,
            -7512.57465309683,
            -5018.31852539635,
            -3551.82629919458,
            -3000.47980521376,
            -2436.41725067266,
            -1305.36051838710,
            -346.558408174331]

PRIOR['MU'] = mu
PRIOR['C'] = c

## Geracao do ensemble ##
n = 48
I = n
J = 1
signal2noise = 5
v_fact = 0.1

wavelet = np.array(wavelet_new)

delta = np.zeros([31,1])
delta[np.around(delta.shape[0]/2).astype(int)-1,0] = 1

wavelet = lowPassFilter2(delta,4,40,60) - lowPassFilter2(delta,4,40,6)

G = acoustic_foward_matrix(wavelet,I)

#P = np.matrix('0.7 0.3 0 0; 0.3 0.7 0 0; 0.33 0.33 0.34 0; 0.1 0.1 0.1 0.7')
P = np.matrix('0.7 0.3 0 0; 0.3 0.7 0 0; 0.33 0.33 0.34 0; 0.1 0.1 0.1 0.7')
P = np.array(P)

facies = simulate_markov_chain(P,n,3,1)
print(facies.shape)
mu, log_imp, seismic = facies_forward_model(facies, PRIOR, G, v_fact)

noise = np.random.randn(I-1,1)
noise = noise/np.std(noise)
std_noise = np.std(seismic)/np.sqrt(signal2noise)
noise = noise*std_noise
seismic = seismic + noise.ravel()

fig, axs = plt.subplots(4)
fig.set_dpi(120)
axs[0].imshow(facies.transpose(), aspect='auto')
axs[1].plot(seismic)
axs[2].plot(log_imp)
axs[3].plot(wavelet)
plt.show()

# Data and GAN

In [ ]:
from scipy.ndimage.filters import gaussian_filter

Pver = np.array(np.matrix('0.7 0.3 0 0; 0.3 0.7 0 0; 0.33 0.33 0.34 0; 0.066 0.066 0.066 0.802'))
Phor = np.array(np.matrix('0.4 0.4 0.1 0.1; 0.4 0.4 0.1 0.1; 0.1 0.1 0.6 0.2; 0.1 0.1 0.2 0.6'))

#Simulation grid size

I = 48
J = 48
initial_facies = 3

prior_map = np.ones([I, J, 4])

simulation = simulate_markov_2Dchain(Phor, Pver, prior_map, initial_facies)
ss = gaussian_filter(simulation, sigma=[2, 1])
st = simulation == 3
ss[st] = 3
ss = np.round(gaussian_filter(ss, sigma=[1.2, 1]))

def facies_forward_model_2D(facies, PRIOR, G, v_fact):
  seismics = []
  impedances = []
  for j in range(0,J):
    mu, log_imp, seismic = facies_forward_model(facies[:,j], PRIOR, G, v_fact)
    seismics.append(seismic)
    impedances.append(log_imp)

  seismics = np.array(seismics).transpose()
  impedances = np.array(impedances).transpose()
  return seismics, impedances

seis, immp = facies_forward_model_2D(ss, PRIOR, G, v_fact)

fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(10, 6))
axes[0].imshow(ss)
axes[1].imshow(seis, cmap='gray')
axes[2].imshow(immp)
fig.tight_layout()

In [ ]:
def create_train_data(data_size = 512, i = 48, j = 48):
  initial_facies = 3
  prior_map = np.ones([I, J, 4])
  
  data = np.zeros([data_size, i, j])
  for i in range(0, data_size):
    simulation = simulate_markov_2Dchain(Phor, Pver, prior_map, initial_facies)
    ss = gaussian_filter(simulation, sigma=[2, 1])
    st = simulation == 3
    ss[st] = 3
    ss = np.round(gaussian_filter(ss, sigma=[1.2, 1]))
    data[i] = ss
  return data

start_time = time.time()
X_bk = create_train_data()
fig = plt.figure(dpi=150)
fig.colorbar(plt.imshow(X_bk[0]))
plt.axis('off')
plt.show()
#print(X_bk)
#X_bk = np.expand_dims(X_bk.transpose(), axis=2)
print('Data generation time (s): ', (time.time() - start_time))

# Save data

In [ ]:
import numpy as np
import h5py

hf = h5py.File('data_set_2D.h5', 'w')
hf.create_dataset('X', data=X_bk)
hf.close()


# Load data

In [ ]:
import numpy as np
import h5py
hf = h5py.File('data_set_2D.h5', 'r')
X_bk = np.array(hf['X'])
hf.close()
print(X_bk.shape)

# Process data

In [ ]:
X = np.copy(X_bk).astype(dtype='float32')
X /= X.max()
#X = (X - 0.5) / 0.5 # normalize (-1,1)
X = np.expand_dims(X, axis=3)
#X = np.transpose(X, axes=(0,2,1,3))
#X = tf.keras.utils.to_categorical(X)
print(X.shape)
print(X_bk.min(),X_bk.max())
print(X.min(),X.max())

# Convolutional Variational Autoencoder

In [ ]:
from __future__ import print_function, division

from IPython.display import clear_output

import tensorflow as tf
from tensorflow import keras

from tensorflow.keras.datasets import mnist
from tensorflow.keras.layers import Input, Dense, Reshape, Flatten, Dropout
from tensorflow.keras.layers import BatchNormalization, Activation, ZeroPadding2D, LeakyReLU
from tensorflow.keras.layers import UpSampling2D, Conv2D, Conv2DTranspose
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam

import tensorflow_probability as tfp

import matplotlib.pyplot as plt

import sys

import numpy as np

class CVAE_Dense(tf.keras.Model):
    def __init__(self):
        super(CVAE_Dense, self).__init__()
        # Input shape
        self.img_rows = 48
        self.img_cols = 48
        self.channels = 1
        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        self.latent_dim = 144

        self.optimizer = Adam(1e-4)

        self.encoder = self.build_encoder()
        self.decoder = self.build_decoder()

    def encode(self, x):
      mean, logvar = tf.split(self.encoder(x), num_or_size_splits=2, axis=1)
      print("encoder: ", self.encoder(x).shape)
      print("mean: ", mean.shape)
      return mean, logvar

    def decode(self, z, apply_sigmoid=False):
      logits = self.decoder(z)
      if apply_sigmoid:
        probs = tf.sigmoid(logits)
        return probs
      return logits

    def reparameterize(self, mean, logvar):
      eps = tf.random.normal(shape=mean.shape)
      return eps * tf.exp(logvar * .5) + mean

    @tf.function
    def sample(self, eps=None):
      if eps is None:
        eps = tf.random.normal(shape=(25, self.latent_dim))
      return self.decode(eps, apply_sigmoid=True)

    def log_normal_pdf(self, sample, mean, logvar, raxis=1):
      log2pi = tf.math.log(2. * np.pi)
      return tf.reduce_sum(-.5 * ((sample - mean) ** 2. * tf.exp(-logvar) + logvar + log2pi), axis=raxis)

    def compute_loss(self, x):
      mean, logvar = self.encode(x)
      z = self.reparameterize(mean, logvar)
      x_logit = self.decode(z)
      cross_ent = tf.nn.sigmoid_cross_entropy_with_logits(logits=x_logit, labels=x)
      logpx_z = -tf.reduce_sum(cross_ent, axis=[1, 2, 3])
      logpz = self.log_normal_pdf(z, 0., 0.)
      logqz_x = self.log_normal_pdf(z, mean, logvar)
      return -tf.reduce_mean(logpx_z + logpz - logqz_x)

    def build_encoder(self):

        model = Sequential()
        model.add(Input(shape=self.img_shape))
        model.add(Conv2D(32, kernel_size=3, strides=2, activation="relu", padding="same"))
        model.add(Conv2D(64, kernel_size=3, strides=2, activation="relu", padding="same"))
        model.add(Conv2D(128, kernel_size=3, strides=2, activation="relu", padding="same"))
        model.add(Flatten())
        model.add(Dense(self.latent_dim + self.latent_dim))
        model.summary()

        return model

    def build_decoder(self):

        model = Sequential()
        model.add(Input(shape=(self.latent_dim,)))
        model.add(Dense(6 * 6 * 32, activation="relu"))
        model.add(Reshape((6, 6, 32)))
        model.add(Conv2DTranspose(128, kernel_size=3, strides=2, padding="same", activation="relu"))
        model.add(Conv2DTranspose(64, kernel_size=3, strides=2, padding="same", activation="relu"))
        model.add(Conv2DTranspose(32, kernel_size=3, strides=2, padding="same", activation="relu"))
        #No activation
        model.add(Conv2DTranspose(1, kernel_size=3, strides=1, padding="same"))

        model.summary()

        return model

    @tf.function
    def train_step(self, x):
      print(self.trainable_variables)
      with tf.GradientTape() as tape:
        loss = self.compute_loss(x)
      gradients = tape.gradient(loss, self.trainable_variables)
      self.optimizer.apply_gradients(zip(gradients, self.trainable_variables))

    def train(self, epochs, batch_size=128, save_interval=50):

        X_train = X
        
        noise = tf.random.normal(shape=[batch_size, self.latent_dim])

        for epoch in range(epochs):

            idx = np.random.randint(0, X_train.shape[0], batch_size)
            imgs = X_train[idx]

            start_time = time.time()
            #for img in imgs:
            self.train_step(imgs)
            end_time = time.time()

            if epoch % save_interval == 0:
              loss = tf.keras.metrics.Mean()
              #for img in imgs:
              loss(self.compute_loss(imgs))
              elbo = -loss.result()
              clear_output(wait=True)
              print('Epoch: {}, Test set ELBO: {}, time elapse for current epoch: {}'
                    .format(epoch, elbo, end_time - start_time))
              self.save_imgs(epoch, X_train[np.random.randint(0, X_train.shape[0], 16)])

    def save_imgs(self, epoch, test_sample):
      mean, logvar = self.encode(test_sample)
      z = self.reparameterize(mean, logvar)
      z = tf.random.normal(shape=[16, self.latent_dim])
      print(z.shape)
      predictions = self.sample(z)
      fig = plt.figure(figsize=(10, 10))

      for i in range(predictions.shape[0]):
        plt.subplot(4, 4, i + 1)
        plt.imshow(np.around(predictions[i, :, :, 0]*3))
        plt.axis('off')

      # tight_layout minimizes the overlap between 2 sub-plots
      #plt.savefig('image_at_epoch_{:04d}.png'.format(epoch))
      plt.show()

    def plot_latent_images(self, n=20, digit_size=48):
      """Plots n x n digit images decoded from the latent space."""

      norm = tfp.distributions.Normal(0, 1)
      grid_x = norm.quantile(np.linspace(0.05, 0.95, n))
      grid_y = norm.quantile(np.linspace(0.05, 0.95, n))
      image_width = digit_size*n
      image_height = image_width
      image = np.zeros((image_height, image_width))

      for i, yi in enumerate(grid_x):
        for j, xi in enumerate(grid_y):
          z = np.array([[xi, yi]])
          x_decoded = self.sample(z)
          digit = tf.reshape(x_decoded[0], (digit_size, digit_size))
          image[i * digit_size: (i + 1) * digit_size,
                j * digit_size: (j + 1) * digit_size] = digit.numpy()

      plt.figure(figsize=(10, 10))
      plt.imshow(np.around(image*3))
      plt.axis('Off')
      plt.show()



cvae_dense = CVAE_Dense()
cvae_dense.train(epochs=1001, batch_size=128, save_interval=500)

# CVAE Fully Conv

In [ ]:
from __future__ import print_function, division

from IPython.display import clear_output

import tensorflow as tf
from tensorflow import keras

from tensorflow.keras.datasets import mnist
from tensorflow.keras.layers import Input, Dense, Reshape, Flatten, Dropout
from tensorflow.keras.layers import BatchNormalization, Activation, ZeroPadding2D, LeakyReLU
from tensorflow.keras.layers import UpSampling2D, Conv2D, Conv2DTranspose
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam

import tensorflow_probability as tfp

import matplotlib.pyplot as plt

import sys

import numpy as np

class CVAE(tf.keras.Model):
    def __init__(self):
        super(CVAE, self).__init__()
        # Input shape
        self.img_rows = 48
        self.img_cols = 48
        self.channels = 1
        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        self.latent_dim = 12
        self.latent_dim_shape = (self.latent_dim, self.latent_dim, self.channels)

        self.optimizer = Adam(1e-4)

        self.encoder = self.build_encoder()
        self.decoder = self.build_decoder()

    def encode(self, x):
      mean, logvar = tf.split(self.encoder(x), num_or_size_splits=2, axis=-1)
      return mean, logvar

    def decode(self, z, apply_sigmoid=False):
      logits = self.decoder(z)
      if apply_sigmoid:
        probs = tf.sigmoid(logits)
        return probs
      return logits

    def reparameterize(self, mean, logvar):
      eps = tf.random.normal(shape=mean.shape)
      return eps * tf.exp(logvar * .5) + mean

    @tf.function
    def sample(self, eps=None):
      if eps is None:
        eps = tf.random.normal(shape=(25, self.latent_dim, self.latent_dim, self.channels))
      return self.decode(eps, apply_sigmoid=True)

    def log_normal_pdf(self, sample, mean, logvar, raxis=1):
      log2pi = tf.math.log(2. * np.pi)
      return tf.reduce_sum(-.5 * ((sample - mean) ** 2. * tf.exp(-logvar) + logvar + log2pi), axis=raxis)

    def compute_loss(self, x):
      mean, logvar = self.encode(x)
      print(mean.shape)
      z = self.reparameterize(mean, logvar)
      x_logit = self.decode(z)
      cross_ent = tf.nn.sigmoid_cross_entropy_with_logits(logits=x_logit, labels=x)
      logpx_z = -tf.reduce_sum(cross_ent, axis=[1, 2, 3])
      logpz = self.log_normal_pdf(z, 0., 0.)
      logqz_x = self.log_normal_pdf(z, mean, logvar)
      return -tf.reduce_mean(logpx_z + logpz - logqz_x)

    def build_encoder(self):

        model = Sequential()
        model.add(Input(shape=self.img_shape))
        model.add(Conv2D(32, kernel_size=3, strides=2, activation="relu", padding="same")) # size 24
        model.add(Conv2D(64, kernel_size=3, strides=1, activation="relu", padding="same")) # size 12
        model.add(Conv2D(128, kernel_size=3, strides=2, activation="relu", padding="same")) # size 6
        model.add(Conv2D(128, kernel_size=3, strides=1, activation="relu", padding="same")) # size 6
        #model.add(Flatten())
        #model.add(Dense(self.latent_dim + self.latent_dim))
        model.add(Conv2D(2, kernel_size=1, strides=1, padding="same"))
        model.summary()

        return model

    def build_decoder(self):

        model = Sequential()
        model.add(Input(shape=(self.latent_dim_shape)))
        model.add(Conv2DTranspose(2, kernel_size=1, strides=1, padding="same", activation="relu"))
        #model.add(Dense(6 * 6 * 32, activation="relu"))
        #model.add(Reshape((6, 6, 32)))
        model.add(Conv2DTranspose(128, kernel_size=3, strides=1, padding="same", activation="relu")) # size 6
        model.add(Conv2DTranspose(128, kernel_size=3, strides=2, padding="same", activation="relu")) # size 12
        model.add(Conv2DTranspose(64, kernel_size=3, strides=1, padding="same", activation="relu")) # size 24
        model.add(Conv2DTranspose(32, kernel_size=3, strides=2, padding="same", activation="relu")) #size 48
        #No activation
        model.add(Conv2DTranspose(1, kernel_size=3, strides=1, padding="same"))

        model.summary()

        return model

    @tf.function
    def train_step(self, x):
      with tf.GradientTape() as tape:
        loss = self.compute_loss(x)
      gradients = tape.gradient(loss, self.trainable_variables)
      self.optimizer.apply_gradients(zip(gradients, self.trainable_variables))

    def train(self, epochs, batch_size=128, save_interval=50):

        X_train = X
        
        noise = tf.random.normal(shape=[batch_size, self.latent_dim, self.latent_dim])

        for epoch in range(epochs):

            idx = np.random.randint(0, X_train.shape[0], batch_size)
            imgs = X_train[idx]

            start_time = time.time()
            #for img in imgs:
            self.train_step(imgs)
            end_time = time.time()

            if epoch % save_interval == 0:
              loss = tf.keras.metrics.Mean()
              #for img in imgs:
              loss(self.compute_loss(imgs))
              elbo = -loss.result()
              clear_output(wait=True)
              print('Epoch: {}, Test set ELBO: {}, time elapse for current epoch: {}'
                    .format(epoch, elbo, end_time - start_time))
              self.save_imgs(epoch, X_train[np.random.randint(0, X_train.shape[0], 16)])

    def save_imgs(self, epoch, test_sample):
      mean, logvar = self.encode(test_sample)
      z = self.reparameterize(mean, logvar)
      z = tf.random.normal(shape=[16,self.latent_dim, self.latent_dim, 1])
      print(z.shape, mean.shape)
      fig, axs = plt.subplots(1,2)
      axs[0].hist(mean.numpy().flatten())
      axs[0].set_title('mean true')
      axs[1].hist(z.numpy().flatten())
      axs[1].set_title('z from normal')
      plt.show()
      predictions = self.sample(eps=z)
      fig = plt.figure(figsize=(10, 10))

      for i in range(predictions.shape[0]):
        plt.subplot(4, 4, i + 1)
        plt.imshow(np.around(predictions[i, :, :, 0]*3))
        plt.axis('off')

      # tight_layout minimizes the overlap between 2 sub-plots
      #plt.savefig('image_at_epoch_{:04d}.png'.format(epoch))
      plt.show()

    def plot_latent_images(self, n=20, digit_size=48):
      """Plots n x n digit images decoded from the latent space."""

      norm = tfp.distributions.Normal(0, 1)
      grid_x = norm.quantile(np.linspace(0.05, 0.95, n))
      grid_y = norm.quantile(np.linspace(0.05, 0.95, n))
      image_width = digit_size*n
      image_height = image_width
      image = np.zeros((image_height, image_width))

      for i, yi in enumerate(grid_x):
        for j, xi in enumerate(grid_y):
          z = np.array([[xi, yi]])
          x_decoded = self.sample(z)
          digit = tf.reshape(x_decoded[0], (digit_size, digit_size))
          image[i * digit_size: (i + 1) * digit_size,
                j * digit_size: (j + 1) * digit_size] = digit.numpy()

      plt.figure(figsize=(10, 10))
      plt.imshow(np.around(image*3))
      plt.axis('Off')
      plt.show()



cvae = CVAE()
cvae.train(epochs=1001, batch_size=128, save_interval=500)

In [ ]:
cvae.encoder.summary()

In [ ]:
cvae.decoder.summary()

# Tests CVAE

In [ ]:
norm = tfp.distributions.Normal(0, 1)
norm.cdf(1)

In [ ]:
import numpy as np
import tensorflow_probability as tfp

qtd = 50
inputn = tf.random.normal(shape=[1, cvae.latent_dim, cvae.latent_dim, 1])

plt.hist(inputn.numpy().flatten())
plt.show()

outn_dense = cvae_dense.sample(eps=tf.reshape(inputn,[1,-1])).numpy()
im_dense = np.round(outn_dense[0].reshape(48,48)*3)

outn = cvae.sample(eps=inputn).numpy()
im = np.round(outn[0].reshape(48,48)*3)

fig, axs = plt.subplots(1,2, figsize=(10, 6))
axs[0].imshow(im_dense)
axs[0].set_title('CVAE Dense')
axs[1].imshow(im)
axs[1].set_title('CVAE Conv')
plt.show()

qtd = 10

im_test = ss.reshape(1,ss.shape[0], -1, 1)/3

mean, logvar = cvae.encode(im_test)
out = cvae.sample(cvae.reparameterize(mean, logvar))

mean_dense, logvar_dense = cvae_dense.encode(im_test)
out_dense = cvae_dense.sample(cvae_dense.reparameterize(mean_dense, logvar_dense))

fig, axs = plt.subplots(1,2, figsize=(10, 6))
axs[0].hist(mean_dense.numpy().flatten())
axs[0].set_title('CVAE Dense')
axs[1].hist(mean.numpy().flatten())
axs[1].set_title('CVAE Conv')
plt.show()

#fig = plt.figure(dpi=100)
#fig.colorbar(plt.imshow(im_test.reshape(I,J)))
#plt.title('True')
#plt.show()

fig, axs = plt.subplots(1,5, figsize=(15, 8))
axs[0].imshow(im_test.reshape(I,J))
axs[0].set_title('True')
axs[1].imshow(tf.reshape(out_dense, [n,n]))
axs[1].set_title('CVAE Dense')
axs[2].imshow(tf.reshape(out,[n,n]))
axs[2].set_title('CVAE Conv')
axs[3].imshow(tf.reshape(mean_dense,[cvae.latent_dim,cvae.latent_dim]))
axs[3].set_title('CVAE Dense latent')
axs[4].imshow(tf.reshape(mean,[cvae.latent_dim,cvae.latent_dim]))
axs[4].set_title('CVAE Conv latent')
plt.show()

#plt.figure(dpi=100)
#plt.hist([np.round(out*3).reshape(-1),  
#          X_bk[:qtd].reshape(-1)], label=['GAN', 'True'])
#plt.legend(loc='upper right')
#plt.show()



# CVAE Dense vs CVAE Conv

In [ ]:
n = 48

In [ ]:
import time

im_test = ss.reshape(1,ss.shape[0], -1, 1)/3

mean_dense, logvar_dense = cvae_dense.encode(im_test)
z_dense = tf.Variable(cvae_dense.reparameterize(mean_dense, logvar_dense))

mean, logvar = cvae.encode(im_test)
z = tf.Variable(cvae.reparameterize(mean, logvar))

shape_dense = [0,0]
shape = [0,0,0,0]

scalar = 0.5

for i in range(0,21):
    clear_output(wait=True)

    z_dense[shape_dense].assign(z_dense[shape_dense] + scalar)
    z[shape].assign(z[shape] + scalar)

    out_dense = cvae_dense.sample(z_dense)
    out = cvae.sample(z)

    fig, axs = plt.subplots(1,2, figsize=(10, 6))
    axs[0].imshow(tf.reshape(out_dense, [n,n]))
    axs[1].imshow(tf.reshape(out,[n,n]))
    fig.suptitle(('Itr ' + str(i)), fontsize=16)
    plt.show()
    time.sleep(2)

# Save models

In [ ]:
#Dense
cvae_dense.encoder.save('cvae_dense_encoder')
cvae_dense.decoder.save('cvae_dense_decoder')

In [ ]:
#Fully Conv
cvae.encoder.save('cvae_encoder')
cvae.decoder.save('cvae_decoder')

# Load models

In [ ]:
cvae.encoder = tf.keras.models.load_model('cvae_encoder')
cvae.decoder = tf.keras.models.load_model('cvae_decoder')

In [ ]:
cvae_dense.encoder = tf.keras.models.load_model('cvae_dense_encoder')
cvae_dense.decoder = tf.keras.models.load_model('cvae_dense_decoder')

# Ensemble Smoother CVAE

In [ ]:
def plt_ensamble(modelo_samples):
    w, h = int(math.sqrt(n_samples)), int(math.sqrt(n_samples))    
    fig, axs = plt.subplots(w,h, figsize=(15, 15))
    gan_facies = tf.experimental.numpy.around(cvae.sample(eps=modelo_samples)*3)
    idx = 0
    for i in range(0,w):
        for j in range(0,h):
            axs[i,j].imshow(tf.reshape(gan_facies[idx], [n,n]))
            axs[i,j].axis('off')
            #axs[i,j].set_xticklabels([])
            #axs[i,j].set_yticklabels([])
            #axs[i,j].set_aspect('equal')
            idx = idx + 1
    #fig.tight_layout()
    plt.show()
def plt_ensamble_dense(modelo_samples):
    w, h = int(math.sqrt(n_samples)), int(math.sqrt(n_samples))    
    fig, axs = plt.subplots(w,h, figsize=(15, 15))
    gan_facies = tf.experimental.numpy.around(cvae_dense.sample(eps=modelo_samples[idx])*3)
    idx = 0
    for i in range(0,w):
        for j in range(0,h):
            axs[i,j].imshow(tf.reshape(gan_facies[idx], [n,n]))
            axs[i,j].axis('off')
            #axs[i,j].set_xticklabels([])
            #axs[i,j].set_yticklabels([])
            #axs[i,j].set_aspect('equal')
            idx = idx + 1
    #fig.tight_layout()
    plt.show()
#plt_ensamble(modelo_samples)

In [ ]:
samples_init = tf.Variable(np.random.normal(size=(1, cvae.latent_dim, cvae.latent_dim, 1)), dtype=tf.float32)

plt.hist(samples_init.numpy().flatten())
plt.show()

plt.imshow(tf.reshape(cvae.sample(eps=samples_init), [n,n]))
plt.show()
plt.close()

samples_init = tf.Variable(np.random.normal(size=(1,100)), dtype=tf.float32)

plt.hist(samples_init.numpy().flatten())
plt.show()

plt.imshow(tf.reshape(cvae_dense.sample(eps=samples_init), [n,n]))
plt.show()
plt.close()

In [ ]:
tf.reduce_mean(modelo_samples, 0, )

In [ ]:
import time
import tensorflow as tf
import tensorflow_probability as tfp


n_ensemble = 30
n_samples = 100
cut_factor = 1

#plt.figure(dpi=120)
#plt.plot(wavelet)
#plt.show()

latent_dim = cvae.latent_dim

G = tf.constant(acoustic_foward_matrix(wavelet,I), np.float32)

plt.figure(dpi=120)
plt.imshow(G)
plt.show()

modelo_samples = tf.Variable(tf.random.normal(shape=[n_samples, cvae.latent_dim, cvae.latent_dim, 1]))
sismica_samples = tf.Variable(tf.zeros([n_samples, G.shape[0], G.shape[1]]))
imp_samples =  tf.Variable(tf.zeros([n_samples, G.shape[0]+1, G.shape[1]]))

samples_init = tf.random.normal([1, cvae.latent_dim, cvae.latent_dim, 1])
gan_facies = tf.experimental.numpy.around(cvae.sample(eps=samples_init)*3)

plt.imshow(gan_facies.reshape(n,n))
plt.show()
plt.close()

#fig = plt.figure(figsize=(10, 10))
#for i in range(16):
#   plt.subplot(4, 4, i + 1)
#   plt.imshow(np.around(gan_facies[i, :, :, 0]))
#   plt.axis('off')
#plt.show()
modelo_referencia = tf.constant(ss.reshape(1,n,n,1), dtype=tf.float32)
modelo_referencia_latent = tf.constant(cvae.encode(modelo_referencia/3)[0])
sismica_experimental_tmp, imp_experimental_tmp = facies_forward_model_2D(modelo_referencia.numpy().reshape(n,n),PRIOR,G.numpy(), v_fact)
sismica_experimental = tf.constant(sismica_experimental_tmp.reshape(-1,J), dtype=tf.float32)
imp_experimental = tf.constant(imp_experimental_tmp.reshape(I,J), dtype=tf.float32)

noise = np.random.randn(I-1,J)
noise = noise/np.std(noise)
std_noise = np.std(sismica_experimental)/np.sqrt(signal2noise)
noise = noise*std_noise
sismica_experimental = sismica_experimental + noise

fig, axs = plt.subplots(1,3)
fig.set_dpi(120)
axs[0].imshow(modelo_referencia.numpy().reshape(n,n))
axs[1].imshow(sismica_experimental, cmap='gray')
axs[2].imshow(imp_experimental)
plt.margins(0,0)
plt.show()

C_d = std_noise**2*tf.eye(sismica_experimental.shape[0]*sismica_experimental.shape[1])
plt.imshow(C_d)
plt.show()

inflation_factor = np.linspace(10,1,n_ensemble)
c = np.sum(1/inflation_factor)
inflation_factor = tf.constant(c*inflation_factor, dtype=tf.float32)
#inflation_factor = tf.ones([n_ensemble,1])*10

erro_referencia_samples = tf.Variable(tf.zeros([n_samples]))
erro_referencia_samples_latent = tf.Variable(tf.zeros([n_samples]))

start_time = time.time()

for sample in range(0,n_samples):
    gan_facies = tf.experimental.numpy.around(cvae.sample(eps=modelo_samples[sample:sample+1])*3)

    sismica_experimental_tmp, imp_experimental_tmp = facies_forward_model_2D(np.array(gan_facies.reshape(n,n)),PRIOR,G.numpy(), v_fact)
    imp_samples[sample].assign(imp_experimental_tmp)
    sismica_samples[sample].assign(sismica_experimental_tmp)
    erro_referencia_samples[sample].assign(tf.reduce_mean(tf.math.squared_difference(modelo_referencia/3, gan_facies/3)))
    erro_referencia_samples_latent[sample].assign(tf.reduce_mean(tf.math.squared_difference(modelo_referencia_latent, modelo_samples[sample])))

print('Total init time(s): ', time.time() - start_time)

plt_ensamble(modelo_samples)

plt.hist(tf.reduce_mean(modelo_samples,0).numpy().flatten())
plt.title('Media modelo_samples')
plt.show()

media_ensemble = tf.Variable(tf.zeros([n_ensemble, 1, cvae.latent_dim, cvae.latent_dim, 1]))
erro_sismico = tf.Variable(tf.zeros([n_ensemble-1]))
erro_impedance = tf.Variable(tf.zeros([n_ensemble-1]))
erro_referencia = tf.Variable(tf.zeros([n_ensemble-1]))
erro_referencia_latent = tf.Variable(tf.zeros([n_ensemble-1]))

for ensemble in range (1,n_ensemble):
    start_time_ensemble = time.time()
    erro_sismico[ensemble-1].assign(tf.reduce_mean(tf.math.squared_difference(sismica_experimental, tf.reduce_mean(sismica_samples,0))))
    erro_impedance[ensemble-1].assign(tf.reduce_mean(tf.math.squared_difference(imp_experimental, tf.reduce_mean(imp_samples,0))))
    erro_referencia[ensemble-1].assign(tf.reduce_mean(erro_referencia_samples))
    erro_referencia_latent[ensemble-1].assign(tf.reduce_mean(erro_referencia_samples_latent))
    
    media_ensemble[ensemble-1].assign(tf.reduce_mean(modelo_samples, 0, keepdims=True))

    

    #U_t = tf.transpose(tf.linalg.cholesky(C_d))
    #noise_t = tf.random.normal([U_t.shape[0],1], mean=0.0, stddev=1.0)
    #d_tio = tf.add(sismica_experimental, tf.reshape(tf.math.sqrt(inflation_factor[ensemble])*tf.matmul(U_t, noise_t), sismica_experimental.shape))

    d_tio = tf.add(sismica_experimental, tf.math.sqrt(inflation_factor[ensemble])*tf.random.normal(sismica_experimental.shape)*std_noise)

    fig, axs = plt.subplots(1,2, figsize=(10, 6))
    axs[0].imshow(d_tio)
    axs[0].set_title('d_tio')
    axs[1].imshow(sismica_experimental)
    axs[1].set_title('d')
    plt.show()


    data_diff = tf.reshape(tf.transpose(sismica_samples - tf.reduce_mean(sismica_samples), perm=[0,2,1]),[n_samples,-1])
    #model_diff = tf.reshape(tf.transpose(modelo_samples - tf.reduce_mean(modelo_samples, 0), perm=[0,1,3,2,4]), [n_samples,-1])
    model_diff = tf.reshape(modelo_samples - tf.reduce_mean(modelo_samples), [n_samples,-1])
    
    #C_dd = tfp.stats.covariance(tf.reshape(tf.transpose(sismica_samples, perm=[0,2,1]),[n_samples,-1]))
    C_dd = tf.linalg.matmul(tf.transpose(data_diff), data_diff)/(n_samples-1)
    C_md = tf.linalg.matmul(tf.transpose(model_diff), data_diff)/(n_samples-1)

    fig = plt.figure(dpi=140)
    plt.imshow(C_dd[:200,:200])
    plt.show()

    fig = plt.figure(dpi=140)
    plt.imshow(C_md[:,:200])
    plt.show()
    #C_dd = tf.linalg.matmul(tf.transpose(data_diff), data_diff)/(n_samples-1)
    #C_dd_experimental = tfp.stats.covariance(tf.reshape(sismica_samples,[n_samples,-1]), tf.reshape(sismica_samples,[n_samples,-1]))
    #C_d_experimental = tfp.stats.covariance(tf.reshape(sismica_samples, [sismica_samples.shape[0],-1]))
    
    K = tf.linalg.matmul(C_md, tf.linalg.pinv(C_dd + inflation_factor[ensemble]*C_d, 0.001*tf.reduce_mean(tf.linalg.tensor_diag_part(C_d))))
    
    #fig = plt.figure(dpi=140)
    #fig.colorbar(plt.imshow(K[:,:200]))
    #plt.show()
    print(C_md.shape)
    print(K.shape)

    for sample in range(0,n_samples):
        #K = tf.reshape(tf.linalg.matmul(cmd_inv,tf.reshape(d_tio-sismica_samples[sample],[-1,1])), modelo_samples[sample].shape)
       
        #dtio_dp = tf.reshape(tf.transpose(d_tio-sismica_samples[sample], perm=[1,0]),[-1,1])
        dtio_dp = tf.reshape(d_tio-sismica_samples[sample], [-1,1])
        Ksum = tf.reshape(tf.linalg.matmul(K,dtio_dp), modelo_samples[sample].shape)
        rmss = modelo_samples[sample] + Ksum

        modelo_samples[sample].assign(rmss)

        gan_facies = tf.experimental.numpy.around(cvae.sample(eps=modelo_samples[sample:sample+1])*3)

        sismica_experimental_tmp, imp_experimental_tmp = facies_forward_model_2D(np.array(gan_facies.reshape(n,n)),PRIOR,G.numpy(), v_fact)
        imp_samples[sample].assign(imp_experimental_tmp)
        sismica_samples[sample].assign(sismica_experimental_tmp)
        erro_referencia_samples[sample].assign(tf.reduce_mean(tf.math.squared_difference(modelo_referencia/3, gan_facies/3)))
        erro_referencia_samples_latent[sample].assign(tf.reduce_mean(tf.math.squared_difference(modelo_referencia_latent, modelo_samples[sample])))

    print('Total update time(s): ', time.time() - start_time_ensemble)
    plt_ensamble(modelo_samples)

    mean_ss = tf.reduce_mean(modelo_samples,0, keepdims=True)
    
    fig, axs = plt.subplots(1,2, figsize=(10, 6))
    axs[0].imshow(tf.reshape(mean_ss, [latent_dim, latent_dim]))
    axs[0].set_title('media modelo_samples')
    axs[1].hist(mean_ss.numpy().flatten())
    axs[1].set_title('hist')
    plt.show()

    facies_es = tf.reshape(tf.experimental.numpy.around(cvae.sample(eps=mean_ss)*3), [n,n])
    sismica_es, imp_es = facies_forward_model_2D(facies_es.numpy(),PRIOR,G.numpy(), v_fact)

    fig, axs = plt.subplots(2,3, figsize=(10, 6))
    axs[0,0].imshow(tf.reshape(modelo_referencia, [n,n]))
    axs[0,1].imshow(tf.reshape(sismica_experimental,[n-1,n]), cmap='gray')
    axs[0,2].imshow(imp_experimental)
    axs[1,0].imshow(facies_es)
    axs[1,1].imshow(sismica_es, cmap='gray')
    axs[1,2].imshow(imp_es)
    fig.tight_layout()
    plt.show()
    plt.close()

    fig, axs = plt.subplots(1,3, figsize=(10, 6))
    axs[0].plot(erro_referencia[:ensemble])
    axs[1].plot(erro_sismico[:ensemble])
    axs[2].plot(erro_referencia_latent[:ensemble])
    plt.show()
    plt.close()

print('Total time(s): ', time.time() - start_time)

In [ ]:
C_dd

In [ ]:
C_md.shape

In [ ]:
modelo_samples = tf.Variable(np.random.normal(size=(n_samples, 1, cvae.latent_dim, cvae.latent_dim, 1)), dtype=tf.float32)
plt.hist(modelo_samples[0].numpy().flatten())
plt.show()
plt.hist(np.mean(modelo_samples.numpy()).flatten())
plt.show()
print(np.mean(modelo_samples.numpy()))
print(tf.reduce_mean(modelo_samples))
plt.hist((modelo_samples - tf.reduce_mean(modelo_samples)).numpy().flatten())
plt.show()
plt.hist((modelo_samples - tf.reduce_mean(modelo_samples,0)).numpy().flatten())
plt.show()


In [ ]:
print(C_md.shape, C_dd.shape)
varn=1-1/math.pow(10,2)
print(varn)
s, u, v = tf.linalg.svd(C_dd + inflation_factor[ensemble]*C_d)
print(s)
diagonal = s
for i in range(len(diagonal)):
    if (tf.reduce_sum(diagonal[0:i+1]))/(tf.reduce_sum(diagonal)) > varn:
        diagonal = (diagonal[0:i+1])
        break

print(diagonal)
u=u[:,0:i+1]
v=v[:,0:i+1]
ss = tf.linalg.diag(diagonal**(-1))
K=tf.experimental.numpy.dot(C_md,(tf.experimental.numpy.dot(tf.experimental.numpy.dot(v,ss),(tf.transpose(u)))))
print(K)

In [ ]:
tf.linalg.pinv(C_dd + inflation_factor[ensemble]*C_d, 0.001*tf.reduce_mean(tf.linalg.diag(C_d)))

In [ ]:
print(0.001*tf.reduce_mean(tf.linalg.tensor_diag_part(C_d)))

In [ ]:
K = tf.linalg.matmul(C_md, tf.linalg.inv(C_dd + inflation_factor[ensemble]*C_d))
dtio_dp = tf.reshape(tf.transpose(d_tio-sismica_samples[sample], perm=[1,0]),[-1,1])
Ksum = tf.reshape(tf.linalg.matmul(K,dtio_dp), modelo_samples[sample].shape)
print(Ksum)

In [ ]:
print(C_d.shape)
nnoise = tf.math.sqrt(inflation_factor[ensemble])*tf.random.normal(sismica_experimental.shape)*0.5*std_noise
fig = plt.figure(dpi=100)
fig.colorbar(plt.imshow(sismica_experimental))
plt.show()
fig = plt.figure(dpi=100)
fig.colorbar(plt.imshow(sismica_experimental+nnoise))
plt.show()

In [ ]:
print(model_diff.shape)
data_diff = (sismica_samples - tf.reduce_mean(sismica_samples,0))
print(data_diff.shape)

tf.linalg.matmul(tf.transpose(model_diff), tf.reshape(tf.transpose(data_diff, perm=[0,2,1]),[n_samples,-1]))
plt.imshow(tf.linalg.matmul(tf.transpose(model_diff), tf.reshape(tf.transpose(data_diff, perm=[0,2,1]),[n_samples,-1]))[:,:200])
plt.show()

In [ ]:
print(C_d.shape)
print(sismica_experimental.shape)
R = np.linalg.cholesky(C_d) #Matriz triangular inferior
U = R.T   #Matriz R transposta
p , w =np.linalg.eig(C_d)

aux = tf.repeat(tf.reshape(sismica_experimental,[-1,1]), n_samples, axis=-1)
mean = 0*(tf.transpose(tf.reshape(sismica_experimental,[-1,1])))
print(mean.shape)
nnoise = tf.transpose(np.random.multivariate_normal(mean[0], np.eye(2256), n_samples))
dddd = np.dot(U, nnoise)
d_obs = aux+math.sqrt(10)*dddd
print(d_obs.shape)
plt.imshow(aux[:,:100])
plt.show()

In [ ]:
print(d_obs[:10,:10])
print(sismica_experimental[:10,:10])

In [ ]:
print(C_dd_experimental.shape)
data_diff = sismica_samples - tf.reduce_mean(sismica_samples,0)
model_diff = tf.reshape(modelo_samples - media_ensemble[ensemble-1], [-1,6,6])
ff = tfp.stats.covariance(sismica_samples, sismica_samples, event_axis=None)#/(n_samples-1)
print(ff)
ff2 = tfp.stats.covariance(tf.reshape(sismica_samples,[n_samples,-1]), tf.reshape(sismica_samples,[n_samples,-1]))
#ff2 = tfp.stats.covariance(tf.reshape(modelo_samples,[n_samples,-1]), tf.reshape(sismica_samples,[n_samples,-1]), event_axis=None)
print(ff2)
fig = plt.figure(dpi=100)
fig.colorbar(plt.imshow(ff2))
plt.show()

data_diff_2 = tf.reshape((sismica_samples - tf.reduce_mean(sismica_samples,0)), [n_samples,-1])
ff3 = tf.linalg.matmul(tf.transpose(data_diff_2), data_diff_2)/(n_samples-1)
print(ff3)
fig = plt.figure(dpi=100)
fig.colorbar(plt.imshow(ff3))
plt.show()

ff3_ = tf.experimental.numpy.dot(tf.transpose(data_diff_2), data_diff_2)/(n_samples-1)
print(tf.reduce_sum(ff3-ff3_))
fig = plt.figure(dpi=100)
fig.colorbar(plt.imshow(ff3))
plt.show()

model_diff_2 = tf.reshape(modelo_samples - tf.reduce_mean(modelo_samples,0), [n_samples,-1])
ff4 = tf.linalg.matmul(tf.transpose(model_diff_2), data_diff_2)/(n_samples-1)
print(ff4)
#fig = plt.figure(dpi=100)
#fig.colorbar(plt.imshow(ff4[:,:36]))
#plt.show()

K1 = tf.reshape(tf.linalg.matmul(cmd_inv,tf.reshape(d_tio-sismica_samples[sample],[-1,1])), modelo_samples[sample].shape)
K2 = tf.reshape(tf.experimental.numpy.dot(cmd_inv,tf.reshape(d_tio-sismica_samples[sample],[-1])), modelo_samples[sample].shape)
print(K1-K2)


In [ ]:
tf.repe

In [ ]:
print(C_d_experimental.shape, C_d_experimental)
print(C_d.shape, C_d)
print(inflation_factor[ensemble])
print(modelo_samples[sample])
inv_m = tf.linalg.pinv(C_d_experimental + inflation_factor[ensemble]*C_d)
print(inv_m.shape, inv_m)
print(C_md_experimental, C_md_experimental.shape)
#cmd_inv = tf.linalg.matmul(C_md_experimental,inv_m)
cmd_inv = tf.experimental.numpy.dot(inv_m, tf.reshape(d_tio-sismica_samples[sample],[-1]))
print(cmd_inv.shape, cmd_inv)
K = tf.reshape(tf.experimental.numpy.dot(C_md_experimental,cmd_inv), modelo_samples[sample].shape)
print(K.shape, K)
rmss = modelo_samples[sample] + K
print(rmss)

In [ ]:
#modelo_samples[sample] = modelo_samples[sample] + C_md_experimental.dot(np.linalg.pinv(C_d_experimental + inflation_factor[ensemble]*C_d)).dot((d_tio-sismica_samples[sample]).reshape(-1))
print(C_d_experimental.shape, C_d_experimental)
print(C_d.shape, C_d)
print(inflation_factor[ensemble])
print(modelo_samples[sample])
inv_m = np.linalg.pinv(C_d_experimental + inflation_factor[ensemble]*C_d)
print(inv_m.shape, inv_m)
print(C_md_experimental, C_md_experimental.shape)
cmd_inv = inv_m.dot((d_tio-sismica_samples[sample]).reshape(-1))
print(cmd_inv.shape, cmd_inv)
K = C_md_experimental.dot(cmd_inv)
print(K.shape, K)
rmss = modelo_samples[sample] + K
print(rmss)

In [ ]:
C_md_experimental.dot(np.linalg.pinv(C_d_experimental + inflation_factor[ensemble]*C_d)).dot((d_tio-sismica_samples[sample]).reshape(-1))

In [ ]:
import time
import tensorflow as tf
import tensorflow_probability as tfp


n_ensemble = 10
n_samples = 25
cut_factor = 1

#plt.figure(dpi=120)
#plt.plot(wavelet)
#plt.show()

latent_dim = cvae_dense.latent_dim

G = tf.constant(acoustic_foward_matrix(wavelet,I), np.float32)

plt.figure(dpi=120)
plt.imshow(G)
plt.show()

modelo_samples = tf.Variable(tf.zeros([n_samples, 1, cvae_dense.latent_dim]))
sismica_samples = tf.Variable(tf.zeros([n_samples, G.shape[0], G.shape[1]]))
imp_samples =  tf.Variable(tf.zeros([n_samples, G.shape[0]+1, G.shape[1]]))

samples_init = tf.random.normal([1, cvae_dense.latent_dim])
gan_facies = tf.experimental.numpy.around(cvae_dense.sample(eps=samples_init)*3)

plt.imshow(gan_facies.reshape(n,n))
plt.show()
plt.close()

#fig = plt.figure(figsize=(10, 10))
#for i in range(16):
#   plt.subplot(4, 4, i + 1)
#   plt.imshow(np.around(gan_facies[i, :, :, 0]))
#   plt.axis('off')
#plt.show()
modelo_referencia = tf.constant(ss.reshape(1,n,n,1), dtype=tf.float32)
modelo_referencia_latent = tf.constant(cvae_dense.encode(modelo_referencia/3)[0])
sismica_experimental_tmp, imp_experimental_tmp = facies_forward_model_2D(modelo_referencia.numpy().reshape(n,n),PRIOR,G.numpy(), v_fact)
sismica_experimental = tf.constant(sismica_experimental_tmp.reshape(-1,J), dtype=tf.float32)
imp_experimental = tf.constant(imp_experimental_tmp.reshape(I,J), dtype=tf.float32)

#noise = np.random.randn(I-1,J)
#noise = noise/np.std(noise)
#std_noise = np.std(sismica_experimental)/np.sqrt(signal2noise)
#noise = noise*std_noise
#sismica_experimental = sismica_experimental + noise

fig, axs = plt.subplots(1,3)
fig.set_dpi(120)
axs[0].imshow(modelo_referencia.numpy().reshape(n,n))
axs[1].imshow(sismica_experimental, cmap='gray')
axs[2].imshow(imp_experimental)
plt.margins(0,0)
plt.show()

C_d = std_noise**2*tf.eye(sismica_experimental.shape[0]*sismica_experimental.shape[1])
plt.imshow(C_d)
plt.show()

inflation_factor = np.linspace(50,10,n_ensemble)
c = np.sum(1/inflation_factor)
inflation_factor = tf.constant(c*inflation_factor, dtype=tf.float32)
#inflation_factor = tf.ones([n_ensemble,1])*10

erro_referencia_samples = tf.Variable(tf.zeros([n_samples]))
erro_referencia_samples_latent = tf.Variable(tf.zeros([n_samples]))

start_time = time.time()

for sample in range(0,n_samples):
    #print(modelo_samples[sample])

    modelo_samples[sample].assign(tf.random.normal([1, cvae_dense.latent_dim])*cut_factor)

    #print(modelo_samples[sample])

    gan_facies = tf.experimental.numpy.around(cvae_dense.sample(eps=modelo_samples[sample])*3)
    #print(gan_facies)
    #plt.imshow(gan_facies.reshape(n,n))
    #plt.show()

    sismica_experimental_tmp, imp_experimental_tmp = facies_forward_model_2D(np.array(gan_facies.reshape(n,n)),PRIOR,G.numpy(), v_fact)
    imp_samples[sample].assign(imp_experimental_tmp)
    sismica_samples[sample].assign(sismica_experimental_tmp)
    erro_referencia_samples[sample].assign(tf.reduce_mean(tf.math.squared_difference(modelo_referencia/3, gan_facies/3)))
    erro_referencia_samples_latent[sample].assign(tf.reduce_mean(tf.math.squared_difference(modelo_referencia_latent, modelo_samples[sample])))

print('Total init time(s): ', time.time() - start_time)

plt_ensamble_dense(modelo_samples)

plt.hist(tf.reduce_mean(modelo_samples,0).numpy().flatten())
plt.show()

media_ensemble = tf.Variable(tf.zeros([n_ensemble, 1, cvae_dense.latent_dim]))
erro_sismico = tf.Variable(tf.zeros([n_ensemble-1]))
erro_impedance = tf.Variable(tf.zeros([n_ensemble-1]))
erro_referencia = tf.Variable(tf.zeros([n_ensemble-1]))
erro_referencia_latent = tf.Variable(tf.zeros([n_ensemble-1]))

for ensemble in range (1,n_ensemble):
    start_time_ensemble = time.time()
    erro_sismico[ensemble-1].assign(tf.reduce_mean(tf.math.squared_difference(sismica_experimental, tf.reduce_mean(sismica_samples,0))))
    erro_impedance[ensemble-1].assign(tf.reduce_mean(tf.math.squared_difference(imp_experimental, tf.reduce_mean(imp_samples,0))))
    erro_referencia[ensemble-1].assign(tf.reduce_mean(erro_referencia_samples))
    erro_referencia_latent[ensemble-1].assign(tf.reduce_mean(erro_referencia_samples_latent))
    
    #U_t = tf.transpose(tf.linalg.cholesky(C_d))
    #noise_t = tf.random.normal([U_t.shape[0],1], mean=0.0, stddev=1.0)
    #d_tio = tf.add(sismica_experimental, tf.reshape(tf.math.sqrt(inflation_factor[ensemble])*tf.matmul(U_t, noise_t), sismica_experimental.shape))

    d_tio = tf.add(sismica_experimental, tf.math.sqrt(inflation_factor[ensemble])*tf.random.normal(sismica_experimental.shape)*std_noise)

    fig, axs = plt.subplots(1,2, figsize=(10, 6))
    axs[0].imshow(d_tio)
    axs[0].set_title('d_tio')
    axs[1].imshow(sismica_experimental)
    axs[1].set_title('d')
    plt.show()

    media_ensemble[ensemble-1].assign(tf.reduce_mean(modelo_samples, 0))

    data_diff = tf.reshape(tf.transpose(sismica_samples - tf.reduce_mean(sismica_samples,0), perm=[0,2,1]),[n_samples,-1])
    #model_diff = tf.reshape(tf.transpose(modelo_samples - tf.reduce_mean(modelo_samples, 0), perm=[0,1,3,2,4]), [n_samples,-1])
    model_diff = tf.reshape(modelo_samples - tf.reduce_mean(modelo_samples, 0), [n_samples,-1])
    
    #C_dd = tfp.stats.covariance(tf.reshape(tf.transpose(sismica_samples, perm=[0,2,1]),[n_samples,-1]))
    C_dd = tf.linalg.matmul(tf.transpose(data_diff), data_diff)/(n_samples-1)
    C_md = tf.linalg.matmul(tf.transpose(model_diff), data_diff)/(n_samples-1)

    fig = plt.figure(dpi=140)
    plt.imshow(C_dd[:200,:200])
    plt.show()

    fig = plt.figure(dpi=140)
    plt.imshow(C_md[:,:200])
    plt.show()
    #C_dd = tf.linalg.matmul(tf.transpose(data_diff), data_diff)/(n_samples-1)
    #C_dd_experimental = tfp.stats.covariance(tf.reshape(sismica_samples,[n_samples,-1]), tf.reshape(sismica_samples,[n_samples,-1]))
    #C_d_experimental = tfp.stats.covariance(tf.reshape(sismica_samples, [sismica_samples.shape[0],-1]))
    
    K = tf.linalg.matmul(C_md, tf.linalg.pinv(C_dd + inflation_factor[ensemble]*C_d, 0.001*tf.reduce_mean(tf.linalg.tensor_diag_part(C_d))))
    
    fig = plt.figure(dpi=140)
    fig.colorbar(plt.imshow(K[:,:200]))
    plt.show()

    for sample in range(0,n_samples):
        #K = tf.reshape(tf.linalg.matmul(cmd_inv,tf.reshape(d_tio-sismica_samples[sample],[-1,1])), modelo_samples[sample].shape)
       
        #dtio_dp = tf.reshape(tf.transpose(d_tio-sismica_samples[sample], perm=[1,0]),[-1,1])
        dtio_dp = tf.reshape(d_tio-sismica_samples[sample], [-1,1])
        Ksum = tf.reshape(tf.linalg.matmul(K,dtio_dp), modelo_samples[sample].shape)
        rmss = modelo_samples[sample] + Ksum

        modelo_samples[sample].assign(rmss)

        gan_facies = tf.experimental.numpy.around(cvae_dense.sample(eps=modelo_samples[sample])*3)

        sismica_experimental_tmp, imp_experimental_tmp = facies_forward_model_2D(np.array(gan_facies.reshape(n,n)),PRIOR,G.numpy(), v_fact)
        imp_samples[sample].assign(imp_experimental_tmp)
        sismica_samples[sample].assign(sismica_experimental_tmp)
        erro_referencia_samples[sample].assign(tf.reduce_mean(tf.math.squared_difference(modelo_referencia/3, gan_facies/3)))
        erro_referencia_samples_latent[sample].assign(tf.reduce_mean(tf.math.squared_difference(modelo_referencia_latent, modelo_samples[sample])))

    print('Total update time(s): ', time.time() - start_time_ensemble)
    #facies_es = np.around(cvae.sample(np.mean(modelo_samples,0).reshape(1,cvae.latent_dim))*3).reshape(I,J)
    #print(kinv)
    plt_ensamble_dense(modelo_samples)

    #fig = plt.figure(dpi=100)
    #fig.colorbar(plt.imshow(tf.reshape(tf.reduce_mean(modelo_samples,0), [latent_dim])))
    #plt.show()

    plt.hist(tf.reduce_mean(modelo_samples,0).numpy().flatten())
    plt.show()

    mean_ss = tf.reduce_mean(modelo_samples,0)
    #plt.hist(tf.reshape(modelo_samples,-1))
    #plt.show()
    #print(tf.reduce_mean(modelo_samples))
    facies_es = tf.reshape(tf.experimental.numpy.around(cvae_dense.sample(eps=mean_ss)*3), [n,n])
    sismica_es, imp_es = facies_forward_model_2D(facies_es.numpy(),PRIOR,G.numpy(), v_fact)

    fig, axs = plt.subplots(2,3, figsize=(10, 6))
    axs[0,0].imshow(tf.reshape(modelo_referencia, [n,n]))
    axs[0,1].imshow(tf.reshape(sismica_experimental,[n-1,n]), cmap='gray')
    axs[0,2].imshow(imp_experimental)
    axs[1,0].imshow(facies_es)
    axs[1,1].imshow(sismica_es, cmap='gray')
    axs[1,2].imshow(imp_es)
    fig.tight_layout()
    plt.show()
    plt.close()

    fig, axs = plt.subplots(1,3, figsize=(10, 6))
    axs[0].plot(erro_referencia[:ensemble])
    axs[1].plot(erro_sismico[:ensemble])
    axs[2].plot(erro_referencia_latent[:ensemble])
    plt.show()
    plt.close()

print('Total time(s): ', time.time() - start_time)

In [ ]:
#tf.linalg.matmul(C_md, tf.linalg.pinv(C_dd + inflation_factor[ensemble]*C_d, 0.001*tf.reduce_mean(tf.linalg.tensor_diag_part(C_d))))
modelo_samples = modelo_samples.numpy().reshape(n_samples,-1).T

In [ ]:
modelo_samples.shape

In [ ]:
from HistoryMatching.ES_MDA import ES_MDA

Alpha = np.ones((10),dtype=int)*10
for t in range(len(Alpha)):
    obs = sismica_experimental.numpy().reshape((-1))
    Obs_sim = sismica_samples.numpy().reshape(-1,n_samples)
    Obs_sim=  np.array(Obs_sim).T
    print('Erro ite_',t, ' : ' ,sum(sum(abs(Obs_sim-obs))))    
    m_x = ES_MDA(modelo_samples.shape[1],m_x,obs,Obs_sim,Alpha[t],R,Corr,2)
    #m_f = UpdateStateFacies(m_x,dim_shape[0],dim_shape[1],redeVAE)
    m_f = redeVAE.predict(m_x)
Obs_sim = [((GetFaciesData(m_f[:,i],dim_shape,position,''))) for i in range(m_f.shape[1])]
Obs_sim=  np.array(Obs_sim).T
print('Erro End: ',sum(sum(abs(Obs_sim-obs))))

In [ ]:
plt.hist(tf.reduce_mean(modelo_samples,0).numpy().flatten()/10)
plt.show()

mean_ss = tf.reduce_mean(modelo_samples,0)/10
mean_ss = tf.random.normal([1, cvae_dense.latent_dim])

plt.hist(mean_ss.numpy().flatten())
plt.show()

#print(tf.reduce_mean(modelo_samples))
facies_es = tf.reshape(tf.experimental.numpy.around(cvae_dense.sample(eps=mean_ss)*3), [n,n])
sismica_es, imp_es = facies_forward_model_2D(facies_es.numpy(),PRIOR,G.numpy(), v_fact)

fig, axs = plt.subplots(2,3, figsize=(10, 6))
axs[0,0].imshow(tf.reshape(modelo_referencia, [n,n]))
axs[0,1].imshow(tf.reshape(sismica_experimental,[n-1,n]), cmap='gray')
axs[0,2].imshow(imp_experimental)
axs[1,0].imshow(facies_es)
axs[1,1].imshow(sismica_es, cmap='gray')
axs[1,2].imshow(imp_es)
fig.tight_layout()
plt.show()
plt.close()

fig, axs = plt.subplots(1,3, figsize=(10, 6))
axs[0].plot(erro_referencia[:ensemble])
axs[1].plot(erro_sismico[:ensemble])
axs[2].plot(erro_referencia_latent[:ensemble])
plt.show()
plt.close()

# Tests Fully Conv

In [ ]:
import time
#from cython.parallel cimport prange

n = 48
n_ensemble = 4
n_samples = 36
cut_factor = 1

#plt.figure(dpi=120)
#plt.plot(wavelet)
#plt.show()

latent_dim = cvae.latent_dim

G = acoustic_foward_matrix(wavelet,I)
print(G.shape)

plt.figure(dpi=120)
plt.imshow(G)
plt.show()

modelo_samples = np.zeros([n_samples, cvae.latent_dim*cvae.latent_dim])

sismica_samples = np.zeros([n_samples, G.shape[0], G.shape[1]])

imp_samples = np.zeros([n_samples, G.shape[0]+1, G.shape[1]])

samples_ini = np.random.normal(size=(1, cvae.latent_dim, cvae.latent_dim, 1))
gan_facies = np.around(cvae.sample(samples_ini).numpy()*3)

plt.imshow(gan_facies.reshape(n,n))
plt.show()

modelo_referencia = ss
modelo_referencia_latent = cvae.encode(modelo_referencia.reshape(1,modelo_referencia.shape[0], -1, 1)/3)[0].numpy()
#print(modelo_referencia_latent)

sismica_experimental_tmp, imp_experimental_tmp = facies_forward_model_2D(modelo_referencia,PRIOR,G, v_fact)

sismica_experimental = sismica_experimental_tmp.reshape(-1,J)
imp_experimental = imp_experimental_tmp.reshape(I,J)

#noise = np.random.randn(I-1,J)
#noise = noise/np.std(noise)
#std_noise = np.std(sismica_experimental)/np.sqrt(signal2noise)
#noise = noise*std_noise
#sismica_experimental = sismica_experimental + noise

fig, axs = plt.subplots(1,3)
fig.set_dpi(120)
axs[0].imshow(modelo_referencia)
axs[1].imshow(sismica_experimental, cmap='gray')
axs[2].imshow(imp_experimental)
plt.margins(0,0)
plt.show()

C_d = std_noise**2*np.eye(sismica_experimental.shape[0]*sismica_experimental.shape[1])

inflation_factor = np.ones([n_ensemble,1])
#inflation_factor = np.linspace(50,10,n_ensemble)
c = np.sum(1/inflation_factor)
inflation_factor = c*inflation_factor

erro_simico_samples = np.zeros([n_samples])
erro_referencia_samples = np.zeros([n_samples])
erro_referencia_samples_latent = np.zeros([n_samples])

start_time = time.time()

for sample in range(0,n_samples):
    modelo_samples[sample] = np.random.normal(size=(1, cvae.latent_dim*cvae.latent_dim))*cut_factor

    gan_facies = np.around(cvae.sample(modelo_samples[sample].reshape(1,cvae.latent_dim, cvae.latent_dim,1)).numpy()*3).reshape(I,J)

    sismica_experimental_tmp, imp_experimental_tmp = facies_forward_model_2D(gan_facies,PRIOR,G, v_fact)
    imp_samples[sample] = imp_experimental_tmp
    sismica_samples[sample] = sismica_experimental_tmp
    erro_simico_samples[sample] = (np.subtract(sismica_experimental, sismica_experimental_tmp)**2).sum()/(n*n)
    erro_referencia_samples[sample] = (np.subtract(modelo_referencia, gan_facies)**2).sum()/(n*n)
    erro_referencia_samples_latent[sample] = (np.subtract(modelo_samples[sample], modelo_samples[sample].reshape(modelo_referencia_latent.shape))**2).sum()/(cvae.latent_dim**2)

print('Total init time(s): ', time.time() - start_time)

media_ensemble = np.zeros([n_ensemble, cvae.latent_dim*cvae.latent_dim])
erro_sismico = np.zeros([n_ensemble-1])
erro_impedance = np.zeros([n_ensemble-1])
erro_referencia = np.zeros([n_ensemble-1])
erro_referencia_latent = np.zeros([n_ensemble-1])

plt_ensamble(modelo_samples.reshape([n_samples, 1, cvae.latent_dim, cvae.latent_dim, 1]))

for ensemble in range (1,n_ensemble):
    start_time_ensemble = time.time()
    
    media_ensemble[ensemble-1] = np.mean(modelo_samples)

    d_tio = sismica_experimental + np.sqrt(inflation_factor[ensemble])*np.random.randn(sismica_experimental.shape[0], sismica_experimental.shape[1])*std_noise

    erro_sismico[ensemble-1] = np.mean(erro_simico_samples)

    erro_impedance[ensemble-1] = (np.subtract(imp_experimental, np.mean(imp_samples))**2).sum()/(n*n)

    erro_referencia[ensemble-1] = np.mean(erro_referencia_samples)

    erro_referencia_latent[ensemble-1] = np.mean(erro_referencia_samples_latent)

    C_d_experimental = np.cov(np.transpose(sismica_samples,axes=(0,2,1)).reshape([sismica_samples.shape[0], -1]),rowvar=False)
    fig = plt.figure(dpi=100)
    fig.colorbar(plt.imshow(C_d_experimental[:200,:200]))
    plt.title('Cd')
    plt.show()
    
    C_md_experimental = (modelo_samples - media_ensemble[ensemble-1]).transpose().dot((sismica_samples - np.mean(sismica_samples,0)).reshape(sismica_samples.shape[0],-1))/(n_samples-1)
    fig = plt.figure(dpi=100)
    fig.colorbar(plt.imshow(C_md_experimental.reshape(36,-1)))
    plt.title('Cmd')
    plt.show()
    print((modelo_samples - media_ensemble[ensemble-1]).transpose().shape)
    print((sismica_samples - np.mean(sismica_samples,0)).reshape(sismica_samples.shape[0],-1).shape)

    for sample in range(0,n_samples):

        modelo_samples[sample] = modelo_samples[sample] + C_md_experimental.dot(np.linalg.inv(C_d_experimental + inflation_factor[ensemble]*C_d)).dot((d_tio-sismica_samples[sample]).reshape(-1))

        gan_facies = np.around(cvae.sample(modelo_samples[sample].reshape(1,cvae.latent_dim, cvae.latent_dim, 1)).numpy()*3).reshape(I,J)

        sismica_experimental_tmp, imp_experimental_tmp = facies_forward_model_2D(gan_facies,PRIOR,G, v_fact)
        imp_samples[sample] = imp_experimental_tmp
        sismica_samples[sample] = sismica_experimental_tmp
        erro_simico_samples[sample] = (np.subtract(sismica_experimental, sismica_experimental_tmp)**2).sum()/(n*n)
        erro_referencia_samples[sample] = (np.subtract(modelo_referencia, gan_facies)**2).sum()/(n*n)
        erro_referencia_samples_latent[sample] = (np.subtract(modelo_samples[sample], modelo_samples[sample].reshape(modelo_referencia_latent.shape))**2).sum()/(cvae.latent_dim**2)

    plt_ensamble(modelo_samples.reshape([n_samples, 1, cvae.latent_dim, cvae.latent_dim, 1]))
    facies_es = np.around(cvae.sample(np.mean(modelo_samples,0).reshape(1,cvae.latent_dim, cvae.latent_dim, 1))*3).reshape(I,J)

    fig, axs = plt.subplots(2,3, figsize=(10, 6))
    axs[0,0].imshow(modelo_referencia)
    axs[0,1].imshow(sismica_experimental, cmap='gray')
    axs[0,2].imshow(imp_experimental)
    axs[1,0].imshow(facies_es)
    axs[1,1].imshow(np.mean(sismica_samples,0), cmap='gray')
    axs[1,2].imshow(np.mean(imp_samples,0))
    fig.tight_layout()
    plt.show()
    plt.close()

    fig, axs = plt.subplots(1,3, figsize=(10, 6))
    axs[0].plot(erro_referencia[1:ensemble+1])
    axs[1].plot(erro_sismico[1:ensemble+1])
    axs[2].plot(erro_referencia_latent[1:ensemble+1])
    plt.show()
    plt.close()

print('Total time(s): ', time.time() - start_time)

In [ ]:
fig = plt.figure(figsize=[20,20])
fig.colorbar(plt.imshow(sismica_samples[10]))
plt.show()

In [ ]:
print(sismica_samples.shape)
print(np.transpose(sismica_samples,axes=(0,2,1)).shape)
fig = plt.figure(figsize=[20,20])
fig.colorbar(plt.imshow(np.cov(np.transpose(sismica_samples,axes=(0,2,1)).reshape([sismica_samples.shape[0], -1]),rowvar=False)[:200,:200]))
plt.show()

print((sismica_samples - np.mean(sismica_samples,0)).shape)
#tmp = (modelo_samples - media_ensemble[ensemble-1]).dot((sismica_samples - np.mean(sismica_samples,0)).reshape(sismica_samples.shape[0],-1).transpose())
tmp = np.cov((modelo_samples - media_ensemble[ensemble-1]), np.transpose(sismica_samples - np.mean(sismica_samples,0),axes=(0,2,1)).reshape(sismica_samples.shape[0],-1))
fig = plt.figure(figsize=[20,20])
plt.imshow(tmp)
plt.show()

In [ ]:
print(sismica_samples.shape)
print(np.mean(sismica_samples,0).shape)
fig = plt.figure(dpi=100)
fig.colorbar(plt.imshow(sismica_samples[0]))
plt.show()
fig = plt.figure(dpi=100)
fig.colorbar(plt.imshow(np.mean(sismica_samples,0)))
plt.show()

In [ ]:
plt.plot(erro_sismico[:-1])
plt.show()
plt.plot(erro_referencia[:-1])
plt.show()
plt.plot(erro_referencia_samples[:-1])
plt.show()


# Ensemble Smoother

In [ ]:
import time

n_ensemble = 20
n_samples = 100
cut_factor = 1

#plt.figure(dpi=120)
#plt.plot(wavelet)
#plt.show()

latent_dim = wgan.latent_dim

G = acoustic_foward_matrix(wavelet,I)
print(G.shape)

plt.figure(dpi=120)
plt.imshow(G)
plt.show()

modelo_samples = np.zeros([n_samples, wgan.latent_dim*wgan.latent_dim])

sismica_samples = np.zeros([n_samples, G.shape[0], G.shape[1]])

imp_samples = np.zeros([n_samples, G.shape[0]+1, G.shape[1]])

samples_ini = np.random.normal(size=(1, wgan.latent_dim*wgan.latent_dim))
gan_facies = np.around((0.5*wgan.generator.predict(samples_ini.reshape(1, wgan.latent_dim, wgan.latent_dim, 1))+0.5011)*3)
modelo_referencia = ss

sismica_experimental_tmp, imp_experimental_tmp = facies_forward_model_2D(modelo_referencia,PRIOR,G, v_fact)

sismica_experimental = sismica_experimental_tmp.reshape(-1,J)
imp_experimental = imp_experimental_tmp.reshape(I,J)

#noise = np.random.randn(I-1,J)
#noise = noise/np.std(noise)
#std_noise = np.std(sismica_experimental)/np.sqrt(signal2noise)
#noise = noise*std_noise
#sismica_experimental = sismica_experimental + noise

fig, axs = plt.subplots(1,3)
fig.set_dpi(120)
axs[0].imshow(modelo_referencia)
axs[1].imshow(sismica_experimental, cmap='gray')
axs[2].imshow(imp_experimental)
plt.margins(0,0)
plt.show()

C_d = std_noise**2*np.eye(sismica_experimental.shape[0]*sismica_experimental.shape[1])

inflation_factor = np.ones([n_ensemble,1])
#inflation_factor = np.linspace(50,10,n_ensemble)
c = np.sum(1/inflation_factor)
inflation_factor = c*inflation_factor

erro_referencia_samples = np.zeros([n_samples])

start_time = time.time()

for sample in range(0,n_samples):
    modelo_samples[sample] = np.random.normal(size=(1, wgan.latent_dim*wgan.latent_dim)).reshape(-1)*cut_factor

    gan_facies = np.around((0.5*wgan.generator.predict(modelo_samples[sample].reshape(1,wgan.latent_dim, -1,1))+0.501)*3).reshape(I,J)

    sismica_experimental_tmp, imp_experimental_tmp = facies_forward_model_2D(gan_facies,PRIOR,G, v_fact)
    imp_samples[sample] = imp_experimental_tmp
    sismica_samples[sample] = sismica_experimental_tmp
    erro_referencia_samples[sample] = (np.subtract(modelo_referencia, gan_facies)**2).sum()/n

media_ensemble = np.zeros([n_ensemble, modelo_samples.shape[1]])
erro_sismico = np.zeros([n_ensemble])
erro_impedance = np.zeros([n_ensemble])
erro_referencia = np.zeros([n_ensemble])

for ensemble in range (1,n_ensemble):
    start_time_ensemble = time.time()
    
    media_ensemble[ensemble-1] = np.mean(modelo_samples)

    d_tio = sismica_experimental + np.sqrt(inflation_factor[ensemble])*np.random.randn(sismica_experimental.shape[0], sismica_experimental.shape[1])*std_noise

    erro_sismico[ensemble-1] = (np.subtract(sismica_experimental, np.mean(sismica_samples))**2).sum()

    erro_impedance[ensemble-1] = (np.subtract(imp_experimental, np.mean(imp_samples))**2).sum()

    erro_referencia[ensemble-1] = np.mean(erro_referencia_samples)

    C_d_experimental = np.cov(sismica_samples.reshape([sismica_samples.shape[0], -1]).transpose())
    fig = plt.figure(dpi=100)
    fig.colorbar(plt.imshow(C_d_experimental))
    plt.title('Cd')
    plt.show()
    C_md_experimental = (modelo_samples - media_ensemble[ensemble-1]).transpose().dot((sismica_samples - np.mean(sismica_samples,0)).reshape(sismica_samples.shape[0],-1))/(n_samples-1)

    fig = plt.figure(dpi=100)
    fig.colorbar(plt.imshow(C_md_experimental))
    plt.title('Cmd')
    plt.show()

    for sample in range(1,n_samples):

        modelo_samples[sample] = modelo_samples[sample] + C_md_experimental.dot(np.linalg.pinv(C_d_experimental + inflation_factor[ensemble]*C_d)).dot((d_tio-sismica_samples[sample]).reshape(-1))

        gan_facies = np.around((0.5*wgan.generator.predict(modelo_samples[sample].reshape(1,wgan.latent_dim, wgan.latent_dim,1))+0.501)*3).reshape(I,J)

        sismica_experimental_tmp, imp_experimental_tmp = facies_forward_model_2D(gan_facies,PRIOR,G, v_fact)
        imp_samples[sample] = imp_experimental_tmp
        sismica_samples[sample] = sismica_experimental_tmp
        erro_referencia_samples[sample] = (np.subtract(modelo_referencia, gan_facies)**2).sum()/n

    facies_es = np.around((0.5*wgan.generator.predict(np.mean(modelo_samples,0).reshape(1,wgan.latent_dim, wgan.latent_dim,1))+0.501)*3).reshape(I,J)

    fig, axs = plt.subplots(2,3, figsize=(10, 6))
    axs[0,0].imshow(modelo_referencia)
    axs[0,1].imshow(sismica_experimental)
    axs[0,2].imshow(imp_experimental)
    axs[1,0].imshow(facies_es)
    axs[1,1].imshow(np.mean(sismica_samples,0))
    axs[1,2].imshow(np.mean(imp_samples,0))
    fig.tight_layout()

print('Total time(s): ', time.time() - start_time)

In [ ]:
fig, axs = plt.subplots(2,3, figsize=(10, 6))
axs[0,0].imshow(modelo_referencia)
axs[0,1].imshow(sismica_experimental)
axs[0,2].imshow(imp_experimental)
axs[1,0].imshow(facies_es)
axs[1,1].imshow(np.mean(sismica_samples,0))
axs[1,2].imshow(np.mean(imp_samples,0))
fig.tight_layout()

In [ ]:
fig, axs = plt.subplots(1,6)
#fig.set_size_inches(7,8)
fig.set_dpi(150)
fig.tight_layout(pad=0)
axs[0].imshow(modelo_referencia)
axs[0].axis('off')
axs[1].imshow(facies_es)
axs[1].axis('off')
axs[2].imshow(sismica_experimental)
axs[2].axis('off')
axs[3].imshow(np.mean(sismica_samples,0))
axs[3].axis('off')
axs[4].imshow(imp_experimental)
axs[4].axis('off')
axs[5].imshow(np.mean(imp_samples,0))
axs[5].axis('off')

#axs[0].set_title('True')
#axs[1].set_title('Mean ES')
#axs[2].set_title('True Seis')
#axs[3].set_title('')
plt.show()

In [ ]:
facies_es = np.around((0.5*wgan.generator.predict(np.mean(modelo_samples,0).reshape(1,-1,1))+0.501)*3).reshape(I,J)

In [ ]:
 (C_md_experimental.dot(np.linalg.pinv(C_d_experimental + inflation_factor[ensemble]*C_d)).dot((d_tio-sismica_samples[sample]).reshape(-1))).shape

In [ ]:
modelo_samples[sample] + C_md_experimental.transpose().dot(np.linalg.pinv(C_d_experimental + inflation_factor[ensemble]*C_d)).transpose().dot(d_tio-sismica_samples[sample])

In [ ]:
(C_d_experimental + inflation_factor[ensemble]*C_d)

In [ ]:
(modelo_samples - media_ensemble[ensemble-1]).shape

In [ ]:
np.mean(sismica_samples,0).shape

In [ ]:
fig, axs = plt.subplots(4)
fig.set_dpi(150)
fig.tight_layout(pad=1.5)
axs[0].plot(erro_sismico_8[:-1].flatten(), label='8')
axs[0].plot(erro_sismico_16[:-1].flatten(), label='16')
axs[0].plot(erro_sismico_32[:-1].flatten(), label='32')
axs[0].legend(fontsize=4,loc='lower left')

axs[1].plot(erro_impedance_8[:-1].flatten(), label='8')
axs[1].plot(erro_impedance_16[:-1].flatten(), label='16')
axs[1].plot(erro_impedance_32[:-1].flatten(), label='32')
axs[1].legend(fontsize=4,loc='lower left')

axs[2].plot(erro_referencia_8[:-1].flatten(), label='8')
axs[2].plot(erro_referencia_16[:-1].flatten(), label='16')
axs[2].plot(erro_referencia_32[:-1].flatten(), label='32')
axs[2].legend(fontsize=4,loc='lower left')

axs[3].plot(inflation_factor.reshape(-1)[:-1].flatten())
axs[0].set_title('Seismic Error')
axs[1].set_title('Impedance Error')
axs[2].set_title('Reference Model Error')
axs[3].set_title('Inflation Factor')
plt.margins(0,1)
#plt.savefig('drive/My Drive/Codes/ensemble_smoother/erros_ml_32_2.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
print(modelo_samples.shape)
all_ensemble = np.around((0.5*gan.generator.predict(modelo_samples.reshape(modelo_samples.shape[1],modelo_samples.shape[0],1))+0.501)*3)
plt.figure(dpi=120)
plt.imshow(all_ensemble.reshape(all_ensemble.shape[0:-1])[:,:], aspect='auto')
plt.show()
print(all_ensemble.shape)
for i in range(0,n_samples):
  all_ensemble[i] = np.around((0.5*gan.generator.predict(modelo_samples[:,i].reshape(1,-1,1))+0.501)*3).reshape(-1,1)
gan_facies = np.around((0.5*gan.generator.predict(modelo_samples[:,0].reshape(1,-1,1))+0.501)*3).reshape(-1,1)
print(gan_facies.shape)
plt.figure(figsize=(3,6))
plt.imshow(gan_facies, aspect='auto')
plt.show()
plt.figure(figsize=(3,6))
plt.imshow(all_ensemble[0], aspect='auto')
plt.show()

In [ ]:
print(all_ensemble_8.shape)

In [ ]:
all_ensemble_8 = np.ones([n_samples, n, 1])
all_ensemble_16 = np.ones([n_samples, n, 1])
all_ensemble_32 = np.ones([n_samples, n, 1])

for i in range(0,n_samples):
  all_ensemble_8[i] = np.around((0.5*gan8.predict(modelo_samples_8[:,i].reshape(1,-1,1))+0.501)*3).reshape(-1,1)
  all_ensemble_16[i] = np.around((0.5*gan16.predict(modelo_samples_16[:,i].reshape(1,-1,1))+0.501)*3).reshape(-1,1)
  all_ensemble_32[i] = np.around((0.5*gan32.predict(modelo_samples_32[:,i].reshape(1,-1,1))+0.501)*3).reshape(-1,1)

all_ensemble_facies_8 = all_ensemble_8.reshape(all_ensemble_8.shape[0:-1]).transpose()
all_ensemble_facies_16 = all_ensemble_16.reshape(all_ensemble_16.shape[0:-1]).transpose()
all_ensemble_facies_32 = all_ensemble_32.reshape(all_ensemble_32.shape[0:-1]).transpose()

hist_facies_8 = np.zeros([all_ensemble_facies_8.shape[0],1])
hist_facies_16 = np.zeros([all_ensemble_facies_16.shape[0],1])
hist_facies_32 = np.zeros([all_ensemble_facies_32.shape[0],1])

hist_imp_8 = np.zeros([all_ensemble_facies_8.shape[0],50])
hist_imp_16 = np.zeros([all_ensemble_facies_16.shape[0],50])
hist_imp_32 = np.zeros([all_ensemble_facies_32.shape[0],50])

shift_bin_8 = np.zeros([all_ensemble_facies_8.shape[0],51])
shift_bin_16 = np.zeros([all_ensemble_facies_16.shape[0],51])
shift_bin_32 = np.zeros([all_ensemble_facies_32.shape[0],51])

for i in range(0,all_ensemble_facies_32.shape[0]):
  ftmp = np.histogram(all_ensemble_facies_8[i,:], bins=[0, 1, 2, 3, 4])[0]
  hist_facies_8[i] = np.argmax(ftmp)
  hist_imp_8[i,:] = np.histogram(imp_samples_8[i,:], bins=50, range=[9,10])[0]
  shift_bin_8[i,:] = np.histogram(imp_samples_8[i,:], bins=50)[1]
  
  ftmp = np.histogram(all_ensemble_facies_16[i,:], bins=[0, 1, 2, 3, 4])[0]
  hist_facies_16[i] = np.argmax(ftmp)
  hist_imp_16[i,:] = np.histogram(imp_samples_16[i,:], bins=50, range=[9,10])[0]
  shift_bin_16[i,:] = np.histogram(imp_samples_16[i,:], bins=50)[1]

  ftmp = np.histogram(all_ensemble_facies_32[i,:], bins=[0, 1, 2, 3, 4])[0]
  hist_facies_32[i] = np.argmax(ftmp)
  hist_imp_32[i,:] = np.histogram(imp_samples_32[i,:], bins=50, range=[9,10])[0]
  shift_bin_32[i,:] = np.histogram(imp_samples_32[i,:], bins=50)[1]

plt.figure(dpi=100, figsize=(4,10))
plt.imshow(hist_facies_8, aspect='auto')
plt.show()

#plt.figure(dpi=100, figsize=(4,10))
#plt.imshow(hist_imp, aspect='auto')
#plt.autoscale(False)
#plt.plot(np.flip(imp_experimental),np.linspace(0,128,128))
#plt.imshow(hist_imp, aspect='auto')
#plt.plot(np.flip(np.mean(imp_samples,1)),np.linspace(0,128,128))
#plt.autoscale(False)
#plt.show()

fig, ax_temp = plt.subplots()
fig.set_size_inches((4,20))
ax_temp.imshow(np.flipud(hist_imp_8), aspect='auto', extent=[shift_bin_8.min(),shift_bin_8.max(),0,n], origin='lower')
print(ax_temp.get_xlim())
ax_temp.plot(np.flip(np.mean(imp_samples_8,1)),np.linspace(0,n,n),'r', lw=3)
print(ax_temp.get_xlim())
ax_temp.plot(np.flip(imp_experimental),np.linspace(0,n,n),'y',lw=3)
print(ax_temp.get_xlim())
plt.margins(0,0)
plt.show()
print(shift_bin.min(), shift_bin.max())

fig, ax_temp = plt.subplots()
fig.set_size_inches((4,20))
ax_temp.imshow(np.flipud(hist_imp_16), aspect='auto', extent=[shift_bin_16.min(),shift_bin_16.max(),0,n], origin='lower')
print(ax_temp.get_xlim())
ax_temp.plot(np.flip(np.mean(imp_samples_16,1)),np.linspace(0,n,n),'r', lw=3)
print(ax_temp.get_xlim())
ax_temp.plot(np.flip(imp_experimental),np.linspace(0,n,n),'y',lw=3)
print(ax_temp.get_xlim())
plt.margins(0,0)
plt.show()
print(shift_bin.min(), shift_bin.max())

fig, ax_temp = plt.subplots()
fig.set_size_inches((4,20))
ax_temp.imshow(np.flipud(hist_imp_32), aspect='auto', extent=[shift_bin_32.min(),shift_bin_32.max(),0,n], origin='lower')
print(ax_temp.get_xlim())
ax_temp.plot(np.flip(np.mean(imp_samples_32,1)),np.linspace(0,n,n),'r', lw=3)
print(ax_temp.get_xlim())
ax_temp.plot(np.flip(imp_experimental),np.linspace(0,n,n),'y',lw=3)
print(ax_temp.get_xlim())
plt.margins(0,0)
plt.show()
print(shift_bin.min(), shift_bin.max())

In [ ]:
csfont = {'fontname':'serif'}
fig, axs = plt.subplots(1,8)
fig.set_dpi(300)
axs[0].imshow(modelo_referencia, aspect='auto')
axs[1].imshow(all_ensemble_facies_8, aspect='auto')
axs[2].imshow(all_ensemble_facies_16, aspect='auto')
axs[3].imshow(all_ensemble_facies_32, aspect='auto')
axs[4].imshow(hist_facies, aspect='auto')
axs[5].imshow(facies_es_8, aspect='auto')
axs[6].plot(np.flip(sismica_experimental),np.linspace(0,n-1,n-1), lw=1, label='True')
axs[6].plot(np.flip(np.mean(sismica_samples_8,1)),np.linspace(0,n-1,n-1), 'r', lw=1, label='ES')
axs[6].legend(fontsize=4,loc='lower right')
axs[6].margins(0,0)
axs[7].imshow(np.flipud(hist_imp), aspect='auto', extent=[np.exp(9.0690473),np.exp(9.80372756),0,n], origin='lower')
axs[7].plot(np.flip(np.exp(imp_experimental)),np.linspace(0,n,n), 'y', lw=1, label='True')
axs[7].plot(np.flip(np.exp(np.mean(imp_samples_8,1))),np.linspace(0,n,n),'r',lw=1, label='ES')
axs[7].legend(fontsize=4,loc='lower right')
#plt.margins(0,0)
#fig.tight_layout()

ffsize = 4

axs[0].set_title('Reference Model', size=ffsize, **csfont)
axs[1].set_title('Ensemble 8', size=ffsize, **csfont)
axs[2].set_title('Ensemble 16', size=ffsize, **csfont)
axs[3].set_title('Ensemble 32', size=ffsize, **csfont)
axs[4].set_title('Most Likely Facies', size=ffsize, **csfont)
axs[5].set_title('Latent Space mean', size=ffsize, **csfont)
axs[6].set_title('Seismic', size=ffsize, **csfont)
axs[7].set_title('Impedance', size=ffsize, **csfont)

#axs[0].set_title('(a)', size=8, **csfont)
#axs[1].set_title('(b)', size=8, **csfont)
#axs[2].set_title('(c)', size=8, **csfont)
#axs[3].set_title('(d)', size=8, **csfont)
#axs[4].set_title('(e)', size=8, **csfont)
#axs[5].set_title('(f)', size=8, **csfont)


#axs[0].margins(0,0)
axs[0].tick_params(labelsize=4)
axs[0].get_xaxis().set_visible(False)
axs[1].get_xaxis().set_visible(False)
axs[1].get_yaxis().set_visible(False)
axs[2].get_xaxis().set_visible(False)
axs[2].get_yaxis().set_visible(False)
axs[3].get_xaxis().set_visible(False)
axs[3].get_yaxis().set_visible(False)
axs[4].get_xaxis().set_visible(False)
axs[4].get_yaxis().set_visible(False)
axs[5].get_xaxis().set_visible(False)
axs[5].get_yaxis().set_visible(False)
#axs[2].axis('off')
#axs[3].axis('off')
axs[6].tick_params(labelsize=4)
#axs[4].set_xlim(-0.12,0.12)
axs[7].tick_params(labelsize=4)
axs[6].get_yaxis().set_visible(False)
axs[7].get_yaxis().set_visible(False)
#axs[4].axis('off')
#axs[5].axis('off')
#plt.savefig('drive/My Drive/Codes/ensemble_smoother/es_result_ml_32_2.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
np.log(axs[5].get_xlim())

In [ ]:
csfont = {'fontname':'serif'}
fig, axs = plt.subplots(1,4)
fig.set_dpi(300)
fig.set_size_inches(4,6)
axs[0].imshow(modelo_referencia, aspect='auto')
#axs[1].imshow(all_ensemble_facies, aspect='auto')
#axs[2].imshow(hist_facies, aspect='auto')
#axs[2].imshow(np.mean(all_ensemble_facies,1), aspect='auto')
axs[1].imshow(facies_es, aspect='auto')
axs[2].plot(np.flip(sismica_experimental),np.linspace(0,127,127), lw=1, label='True')
axs[2].plot(np.flip(np.mean(sismica_samples,1)),np.linspace(0,127,127), 'r', lw=1, label='ES')
#axs[4].legend(fontsize=4,loc='lower right')
axs[2].margins(0,0)
# axs[3].set_xlim(np.exp([9.093,9.808]))

axs[3].imshow(np.flipud(hist_imp), aspect='auto', extent=[np.exp(9.135140927930983),np.exp(9.74808053081319),0,128], origin='lower')
axs[3].plot(np.flip(np.exp(np.mean(imp_samples,1))),np.linspace(0,128,128),'r',lw=1, label='ES')
axs[3].plot(np.flip(np.exp(imp_experimental)),np.linspace(0,128,128), 'y', lw=1, label='True')
#axs[3].set_xlim(np.exp(ax_temp.get_xlim()[0]),np.exp(ax_temp.get_xlim()[1]))
#axs[5].legend(fontsize=4,loc='lower right')
#plt.margins(0,0)
#fig.tight_layout()

#axs[0].set_title('Reference Model', size=5, **csfont)
#axs[1].set_title('Ensemble', size=5, **csfont)
#axs[2].set_title('Most Likely Facies', size=5, **csfont)
#axs[3].set_title('Latent Space mean', size=5, **csfont)
#axs[4].set_title('Seismic', size=5, **csfont)
#axs[5].set_title('Impedance', size=5, **csfont)

axs[0].set_title('(a)', size=10, **csfont)
axs[1].set_title('(b)', size=10, **csfont)
axs[2].set_title('(c)', size=10, **csfont)
axs[3].set_title('(d)', size=10, **csfont)
#axs[4].set_title('(e)', size=8, **csfont)
#axs[5].set_title('(f)', size=8, **csfont)


axs[0].axis('off')
#axs[1].axis('off')
#axs[2].axis('off')
axs[1].axis('off')
axs[2].tick_params(labelsize=6)
axs[2].set_xlim(-0.12,0.12)
axs[3].tick_params(labelsize=6)

#axs[3].get_xlim()
axs[2].get_yaxis().set_visible(False)
axs[3].get_yaxis().set_visible(False)
#axs[4].axis('off')
#axs[5].axis('off')
#plt.savefig('drive/My Drive/Codes/ensemble_smoother/es_result_ml.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
qtd = 5000
input_img = np.random.normal(size=(qtd, 16, 1))
output_optim = gan.generator.predict(input_img)
print(output_optim.shape)
fig = plt.figure(dpi=150)
fig.colorbar(plt.imshow(np.round((0.5*output_optim+0.5).reshape(qtd,128)[:100,:41].transpose()*3)))
plt.axis('off')
plt.show()

model_encoder = Sequential()
model_encoder.add(Conv1D(32, kernel_size=3, strides=2, input_shape=gan.img_shape, padding="same"))
model_encoder.add(LeakyReLU(alpha=0.2))
model_encoder.add(Dropout(0.25))
model_encoder.add(Conv1D(64, kernel_size=3, strides=2, padding="same"))
#model.add(ZeroPadding2D(padding=((0,1),(0,1))))
model_encoder.add(BatchNormalization(momentum=0.8))
model_encoder.add(LeakyReLU(alpha=0.2))
model_encoder.add(Dropout(0.25))
model_encoder.add(Conv1D(128, kernel_size=3, strides=2, padding="same"))
model_encoder.add(BatchNormalization(momentum=0.8))
model_encoder.add(LeakyReLU(alpha=0.2))
model_encoder.add(Dropout(0.25))
model_encoder.add(Conv1D(256, kernel_size=3, strides=1, padding="same"))
model_encoder.add(BatchNormalization(momentum=0.8))
model_encoder.add(LeakyReLU(alpha=0.2))
model_encoder.add(Dropout(0.25))
model_encoder.add(Flatten())
model_encoder.add(Dense(16, activation='linear'))

X = output_optim
Y = input_img.reshape(qtd, gan.latent_dim)

model_encoder.compile(loss='mse', optimizer='adam')
start_time = time.time()
model_encoder.fit(X, Y, epochs=20, batch_size=16, validation_split=0.2, verbose=True)
print('Training pos encoder model time (s): ', (time.time() - start_time))

In [ ]:
from random import randint
i = randint(0,X.shape[0])
i = 1331
T = X[i:i+1,:,:]
T[0,:,:] = modelo_referencia
pred_encoder = np.array(model_encoder.predict(T))
print(pred_encoder)
plt.hist(pred_encoder, bins=4)
plt.show()
fake_real = gan.generator.predict(np.expand_dims(pred_encoder, axis=2))
print(fake_real.shape)

out_plot = 0.5*np.array([fake_real[0,:,:], T[0,:,:]])+0.5
print(out_plot.shape)

fig = plt.figure(dpi=150, figsize=(6,6))
fig.colorbar(plt.imshow(np.around(out_plot.reshape(2,128)[:,:].transpose()*3)))
plt.axis('off')
plt.show()

fig = plt.figure(dpi=150, figsize=(6,6))
fig.colorbar(plt.imshow(pred_encoder.transpose()))
plt.axis('off')
plt.show()